# Introduction to the SCOPE Algorithm
**Presenter:** Jenny Lee  &  **Supervisor:** Pathum Kossinna & **Collaborator**: Dr. Cai

![first](image/SCOPE-image.png)

## Why SCOPE?
**Why?**
- Due to the *noisy nature* of -omics data, stability to a machine learning algorithm methods have posed significant problems. 

**How?**
- The SCOPE algorithm poses a solution to aforementioned problem by integrating boostrapped LASSO-regression and co-expression analysis.

**Therefore...**
- The SCOPE algorithm allows us to identify the **core genes** that could be a cause for disease, as well as underlying biological pathways the core genes interact. 

### Project Roadmap
- SCOPE-Stabilized LASSO Regression (bootstrapped LASSO regression)
- Co-expression analysis and differential co-expression analysis
- Pathway Enrichment with Over Representation Analysis
![roadmap](image/SCOPE_Roadmap.png)

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
from scipy.stats import norm
import plotly.express as px
import plotly
import plotly.graph_objects as go
import plotly.subplots as sp
import warnings
from tqdm.notebook import tqdm
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
warnings.filterwarnings('ignore')

### GSE98394 Dataset 
- Retrieved from NCBI: [Gene Expression Omnibus](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE98394). 
- Contains transcriptome data of 78 treatment-naive melanocytic tumor patients. The patients' stage of cancer varies from benign to malignant.  

In [ ]:
raw_data = pd.read_csv("GSE98394_expression.txt", sep= "\t", header=None, skiprows=1)
raw_data.head()

In [ ]:
matrix_data = pd.read_csv("GSE98394_series_matrix.txt", sep= "\t", on_bad_lines="skip", skiprows=50)
matrix_data.head()

In [ ]:
df = pd.read_csv("data-cleaned-for-use/GSE98394_final.csv").drop(["Unnamed: 0"], axis=1)
replace = {" Solid Tissue Normal": "Common acquired nevus", " Primary Tumor": "Primary melanoma"}
df["phen"] = df["phen"].replace(replace, regex=True)

all_phens = df["phen"].unique()
print("We have " + str(len(all_phens)) + " types of unique phenotypes in this dataset. \n" + 
"They are: " + ", ".join([str(x) for x in all_phens]) + ".")

df.head()

### Melanocytic Tumor
On the deepest layer of the epidermis, cells called as **melanocytes** are located. They are responsible for producing the skin's pigment or color. Melanoma is a form of *skin cancer* that extends into the deeper layer of skins. [\[1\]](https://www.mayoclinic.org/diseases-conditions/melanoma/symptoms-causes/syc-20374884)

| Phenotype | Description | 
| :- | :- |
| **Common acquired nevus** | Benign stage of melanocytic tumor; a single or small number of pathogenic mutations occur, with no apprent genomic alterations. [\[2\]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3965271/) |
| **Primary melanoma** | Malignant stage of melanocytic tumor; high number of mutations have occurred, inducing an oppression of tumor-suppression mechanisms and activate additional oncogenic pathways. [\[3\]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4852024/) |

In [ ]:
common_nevus = 0
primary_melanoma = 0
for ind, row in df.iterrows():
    condition = row["phen"]
    if condition == "Common acquired nevus":
        common_nevus += 1
    elif condition == "Primary melanoma":
        primary_melanoma += 1

print("Number of patients with common acquired nevus is: " + str(common_nevus) + ".")
print("Number of patients with primary melanoma is: " + str(primary_melanoma) + ".")

## LASSO-Regression
![lasso](image/lasso_regression.png)

- Use shrinkage and feature selection to reduce over-fitting and generalization. 
- Used 10 k-fold cross-validation to get the optimal value for lambda. 
- Perform boostrapped LASSO regression with an iteration of 1000 times to counter instability of the traditional LASSO. 
### Outcome
Selection of core genes based on number of hits each probe has.

In [ ]:
lasso_summary = pd.read_csv("data-processed-changed/stabilized_lasso_Summary.csv").sort_values(by="model.count", ascending=False)
lasso_summary.rename(columns={"model.count":"model_count", "min.val":"min_val", "max.val":"max_val"}, inplace=True)
lasso_summary.head()

In [ ]:
# construct horizontal bar chart with top 30 hits of model count from lasso_head dataframe 
lasso_top_30 = lasso_summary.head(30)

# set threshold value
threshold_value = 800
threshold_genes = lasso_top_30[lasso_top_30["model_count"] > threshold_value]
print("There are " + str(len(threshold_genes)) + " genes that had greater hits than the threshold value. \n" +
     "They are: " + ", ".join([str(x) for x in threshold_genes["probe"].unique()]) + ".")
threshold_genes

In [ ]:
# rearrange dataframe
lasso_melt = pd.melt(lasso_top_30, id_vars=["probe", "model_count"], value_vars=["min_val", "max_val"])
rename_variable = {"min_val":"Minimum Value", "max_val": "Maximum Value"}
lasso_melt["variable"] = lasso_melt["variable"].replace(rename_variable)
lasso_melt.head()

In [ ]:
def lasso_regression():
    sns.set(style="darkgrid")
    fig, axis = plt.subplots(1,2, sharex=False, figsize=(15, 8))
    fig.suptitle("Boostrapped LASSO Regression Results")
    sp1 = sns.barplot(ax=axis[0], x=lasso_top_30.model_count, y=lasso_top_30.probe, color="lightblue")
    sp1.set_ylabel("Gene Probe", fontsize=16)
    sp1.set_xlabel("Model Count", fontsize=13)
    sp1.axhline(3.55, 0, 3, color="r", ls="--")
    sp2 = sns.barplot(ax=axis[1], data=lasso_melt, x="value", y="probe", hue="variable")
    sp2.set(ylabel=None)
    sp2.set_xlabel("Minimum and Maximum Coefficient Value", fontsize=13)
    sns.despine(left=True, bottom=True)
    for i in range(1, len(axis)):
        axis[i].set_ylim(axis[0].get_ylim())
        axis[i].set_yticks([])
    fig.tight_layout()
    sp2.axhline(3.6, 0, 2, color="r", ls="--")
    fig.text(0.5, 0.001, "Figure 1. Comparing number of counts, minimum value of coefficient, and maximum value of coefficients of top 30 model count genes.", ha="center")

In [ ]:
lasso_regression()

## Co-expression Analysis and Differential Co-expression Analysis
- Perform 100 iterations to obtain a null distribution for the pairwise correlation between randomly selected genes.
### Outcome
Select secondary genes that will form the core gene network with previously selected core genes. Cutoff value for selecting secondary genes are 97.5th percentile of the null distributions.

### Co-expression Analysis Outcome

In [ ]:
corr_null_results = dd.read_csv("large-data/stabilized_lasso_CorrNull-Copy1.csv", blocksize=1e+6)
with ProgressBar():
    corr_null_results = corr_null_results.compute()
corr_null_results.head()

### Differential Co-expression Analysis Outcome

In [ ]:
diffcor_results = dd.read_csv("large-data/stabilized_lasso_DiffCorrNull-Copy1.csv", blocksize=1e+6)
with ProgressBar():
    diffcor_results = diffcor_results.compute()
diffcor_results.head()

In [ ]:
def calculate_mean(df):
    df["Mean"] = df.mean(axis=1)
    return df

diffcor = calculate_mean(diffcor_results)
cor = calculate_mean(corr_null_results)
print(diffcor.Mean)
print(cor.Mean)

In [ ]:
def coexpression_analysis():
    plt.figure(figsize=(14, 7))
    sns.set()
    sns.distplot(diffcor["Mean"], color="skyblue", label="Diff-CoEx", kde=True)
    sns.distplot(cor["Mean"], color="pink", label="CoEx", kde=True)
    plt.title("Correlational and Differential Correlational Analysis Distribution", fontsize=16)
    plt.axvline(cor["Mean"].quantile(0.975), 0, 1, color="r", ls="--")
    plt.axvline(cor["Mean"].quantile(0.025), 0, 1, color="r", ls="--")
    plt.axvline(diffcor["Mean"].quantile(0.975), 0, 1, color="b", ls="--")
    plt.xlabel("Distribution", fontsize=13)
    plt.ylabel("Distribution Density", fontsize=13)
    plt.text(cor["Mean"].quantile(0.975)+0.005, 0.5, "97.5th percentile", color="r", size="medium")
    plt.text(cor["Mean"].quantile(0.025)-0.07, 0.5, "2.5th percentile", color="r", size="medium")
    plt.text(diffcor["Mean"].quantile(0.975)+0.005, 0.5, "97.5th percentile", color="b", size="medium")
    plt.legend()
    plt.text(0.1, -2.5, "Figure 2. Compare null distribution of coexpression analysis and differential coexpression analysis, with a mark of 97.5th percentile.", ha="center")
    plt.show()

In [ ]:
coexpression_analysis()

In [ ]:
print("----Cutoff Value----")
print("The positive 97.5 percentile for coexpression analysis is " + str(cor["Mean"].quantile(0.975)))
print("The negative 97.5 percentile for coexpression analysis is " + str(cor["Mean"].quantile(0.025)))

print("The absolute 97.5 percentile for differential coexpression analysis is " + str(diffcor["Mean"].quantile(0.975)))

## Pathway Enrichment
- Aim to see whether a particular set of genes overlap with known biological pathways.
- Used **Kyoto Encyclopedia of Genes and Genomes (KEGG)** database to obtain information regarding biological pathways.
- **Over Representation Analysis** was used to verify statistical significance and **WebGastaltR** package was used to test pathway enrichment against the KEGG databse. 
### Outcome
Discovers pathways that are commonly influenced by the core gene network. 

In [ ]:
secondary_genes = pd.read_csv("data-processed-changed/stabilized_lasso_SecondaryGenes.csv")
secondary_genes.head()

In [ ]:
pathways = pd.read_csv("data-processed-changed/GSE98394_Pathways.csv")
pathways.head()

In [ ]:
cleaned_pathways = pd.read_csv("data-final-outcomes/csv/GSE98394_sheet2.csv")
cleaned_pathways.head()

**Pathway Overlap Score** has a maximum value of 4. Based on our findings, galactose metabolism has the highest POS. 

In [ ]:
def create_pivot_tables(gene_pathway):
    for ind, row in cleaned_pathways.iterrows():
        if row["Pathway Name"] == gene_pathway:
            userId = row["GeneSet"]
            
    pathway_isolation = pathways[pathways["geneSet"] == userId]["userId"].tolist()
    pathway_list = " ".join(pathway_isolation).split(";")

    col_names = ["Core Gene", "Secondary Gene", "Tumor Correlation", "Normal Correlation"]
    df = pd.DataFrame(columns=col_names)

    for index, row in secondary_genes.iterrows():
        if row["Secondary Gene"] in pathway_list:
            newdict = {"Core Gene":row["Core Gene Name"],
                      "Secondary Gene":row["Secondary Gene"],
                      "Tumor Correlation":row["Tumor Correlation"],
                      "Normal Correlation":row["Normal Correlation"]}
            df = df.append(newdict, ignore_index=True)  
            
    df["Core Gene"] = df["Core Gene"].replace(np.nan, "NaN", regex=True)
    return df

create_pivot_tables("Galactose metabolism").head()

In [ ]:
def create_heatmaps(gene_pathway):
    df = create_pivot_tables(gene_pathway)
    
    tumor_result = df.pivot(index="Secondary Gene", columns="Core Gene", values="Tumor Correlation")
    tumor_result.fillna(0, inplace=True)

    normal_result = df.pivot(index="Secondary Gene", columns="Core Gene", values="Normal Correlation")
    normal_result.fillna(0, inplace=True)

    diff_result = df
    diff_result["Difference"] = abs(diff_result["Tumor Correlation"] - diff_result["Normal Correlation"])
    diff_result = diff_result.pivot(index="Secondary Gene", columns="Core Gene", values="Difference")
    diff_result.fillna(0, inplace=True)
    
    fig, ax = plt.subplots(1,3, sharey=True, figsize=(12,10))
    map1 = sns.heatmap(tumor_result, cbar=False, linewidths=1, square=False, cmap="PiYG", annot = True, annot_kws={"size":7}, center=0, ax=ax[0])
    map2 = sns.heatmap(normal_result, linewidths=1, square=False, cmap="PiYG", annot = True, annot_kws={"size":7}, center=0, ax=ax[1])
    map3 = sns.heatmap(diff_result, linewidths=1, square=False, cmap="PuOr", annot = True, annot_kws={"size":7}, center=0, ax=ax[2])
    map2.set_ylabel("")
    map3.set_ylabel("")
    map1.set_xlabel("")
    map2.set_xlabel("Core Gene", fontsize=14)
    map3.set_xlabel("")
    map1.set_title("")
    map2.set_title("Normal Correlation")
    map3.set_title("Correlation Difference")
    plt.suptitle("Pathway Name: " + str(gene_pathway), y=0.925)
    fig.text(0.5, 0.05, "Figure 3. Heatmap based on correlation of tumor, normal, and absolute difference of a specific pathway.", ha="center")
    plt.show()

In [ ]:
create_heatmaps("Lysosome")

In [ ]:
# creating interactive figure; step1. rearrange dataframe 
all_path = cleaned_pathways["Pathway Name"].unique()

col_names = ["Pathway Name", "Core Gene", "Secondary Gene", "Tumor Correlation", "Normal Correlation"]
final_df = pd.DataFrame(columns=col_names)

for pathway in all_path:
    for ind, row in cleaned_pathways.iterrows():
        if row["Pathway Name"] == pathway:
            userId = row["GeneSet"]
            
    pathway_isolation = pathways[pathways["geneSet"] == userId]["userId"].tolist()
    pathway_list = " ".join(pathway_isolation).split(";")

    for index, row in secondary_genes.iterrows():
        if row["Secondary Gene"] in pathway_list:
            newdict = {"Pathway Name":pathway,
                       "Core Gene":row["Core Gene Name"],
                       "Secondary Gene":row["Secondary Gene"],
                       "Tumor Correlation":row["Tumor Correlation"],
                       "Normal Correlation":row["Normal Correlation"]}
            final_df = final_df.append(newdict, ignore_index=True)  

final_df.head()

In [ ]:
# creating interactive figure; step2. visualizing the figure
interactive_fig = sp.make_subplots(rows=1, cols=4,
                                   subplot_titles=("Tumor", "Normal", "", "Difference"),
                                  column_widths=[0.33, 0.33, 0.01, 0.33])

pathways_list = cleaned_pathways["Pathway Name"].unique()
firstvis = lambda x: True if (x=="Galactose metabolism") else False

pathway_buttons = []
currnum = 0

# design a function to automatically generate heatmaps
def create_plotly_heatmap(dataframe, correlation, pathway, hovertemplate):
    df = dataframe.loc[dataframe["variable"] == correlation]
    fig = go.Heatmap(z=df["value"], x=df["Core Gene"], y=df["Secondary Gene"], visible=firstvis(pathway),
                     name=correlation, coloraxis= "coloraxis" if correlation != "Difference" else "coloraxis2",
                     customdata=np.stack((df["Core Gene"], 
                                          df["Secondary Gene"],
                                          df["value"],
                                          df["Pathway"]), axis=1),
                     hovertemplate=hovertemplate, hoverinfo="x", hoverongaps=False)
    return fig

# create heatmaps
for pathway in pathways_list:
    df = final_df[final_df["Pathway Name"] == pathway]
    df["Core Gene"] = df["Core Gene"].replace(np.nan, "ENSG00000152022", regex=True)
    df["Difference"] = abs(df["Tumor Correlation"] - df["Normal Correlation"])
    df_melt = pd.melt(df, id_vars=["Core Gene", "Secondary Gene"], value_vars=["Tumor Correlation", "Normal Correlation", "Difference"])
    df_melt["Pathway"] = pathway
    
    hovertemplate = "<br>".join(["<b>Core Gene:  %{customdata[0]}</b>",
                                 "Secondary Gene: %{customdata[1]}",
                                 "Pathway: %{customdata[3]}",
                                 "Correlation Value: %{customdata[2]}"])

    tumor_trace = create_plotly_heatmap(df_melt, "Tumor Correlation", pathway, hovertemplate)
    normal_trace = create_plotly_heatmap(df_melt, "Normal Correlation", pathway, hovertemplate)
    diff_trace = create_plotly_heatmap(df_melt, "Difference", pathway, hovertemplate)
    
    interactive_fig.append_trace(tumor_trace, 1,1)
    interactive_fig.append_trace(normal_trace, 1,2)
    interactive_fig.append_trace(diff_trace, 1,4)

# create buttons
for pathway in pathways_list:
    global currnum
    traces = [False] * len(interactive_fig.data)
    oldnum = currnum
    currnum += 3
    traces[0:oldnum] = [False for i in traces[0:oldnum]]
    traces[oldnum:currnum] = [True for i in traces[oldnum:currnum]]
    pathway_buttons.append(dict(label=str(pathway),
                               method="update", args=[{"visible":traces}]))

interactive_fig.update_layout(updatemenus=[dict(active=0,
                                                buttons=pathway_buttons,
                                                direction="down", pad={"r": 0, "t": 10},
                                                showactive=True, x=0, xanchor="left", y=1.14, yanchor="top")],
                             showlegend=False, width=700, height=700, title_text= "Compare Correlation", title_x=0.6, title_y=0.999)

interactive_fig.update_layout(coloraxis = {'colorscale':"RdBu_r", 'showscale':True}, coloraxis2 = {'colorscale':"Reds", 'showscale':True})
interactive_fig.layout.coloraxis.colorbar.x = 0.6
interactive_fig.layout.coloraxis.colorbar.thickness = 15
interactive_fig.layout.coloraxis.colorbar.tickfont = dict(size=10)
interactive_fig.layout.coloraxis2.colorbar.x = 0.99
interactive_fig.layout.coloraxis2.colorbar.thickness = 15
interactive_fig.layout.coloraxis2.colorbar.tickfont = dict(size=10)
interactive_fig.update_yaxes(showticklabels=False, row=1, col=2)
interactive_fig.update_yaxes(showticklabels=False, row=1, col=4)

In [ ]:
def interactive_visualization():
    # creating interactive figure; step2. visualizing the figure
    interactive_fig = sp.make_subplots(rows=1, cols=4,
                                       subplot_titles=("Tumor", "Normal", "", "Difference"),
                                      column_widths=[0.33, 0.33, 0.01, 0.33])

    pathways_list = cleaned_pathways["Pathway Name"].unique()
    firstvis = lambda x: True if (x=="Galactose metabolism") else False

    pathway_buttons = []
    currnum = 0

    # design a function to automatically generate heatmaps
    def create_plotly_heatmap(dataframe, correlation, pathway, hovertemplate):
        df = dataframe.loc[dataframe["variable"] == correlation]
        fig = go.Heatmap(z=df["value"], x=df["Core Gene"], y=df["Secondary Gene"], visible=firstvis(pathway),
                         name=correlation, coloraxis= "coloraxis" if correlation != "Difference" else "coloraxis2",
                         customdata=np.stack((df["Core Gene"], 
                                              df["Secondary Gene"],
                                              df["value"],
                                              df["Pathway"]), axis=1),
                         hovertemplate=hovertemplate, hoverinfo="x", hoverongaps=False)
        return fig

    # create heatmaps
    for pathway in pathways_list:
        df = final_df[final_df["Pathway Name"] == pathway]
        df["Core Gene"] = df["Core Gene"].replace(np.nan, "ENSG00000152022", regex=True)
        df["Difference"] = abs(df["Tumor Correlation"] - df["Normal Correlation"])
        df_melt = pd.melt(df, id_vars=["Core Gene", "Secondary Gene"], value_vars=["Tumor Correlation", "Normal Correlation", "Difference"])
        df_melt["Pathway"] = pathway

        hovertemplate = "<br>".join(["<b>Core Gene:  %{customdata[0]}</b>",
                                     "Secondary Gene: %{customdata[1]}",
                                     "Pathway: %{customdata[3]}",
                                     "Correlation Value: %{customdata[2]}"])

        tumor_trace = create_plotly_heatmap(df_melt, "Tumor Correlation", pathway, hovertemplate)
        normal_trace = create_plotly_heatmap(df_melt, "Normal Correlation", pathway, hovertemplate)
        diff_trace = create_plotly_heatmap(df_melt, "Difference", pathway, hovertemplate)

        interactive_fig.append_trace(tumor_trace, 1,1)
        interactive_fig.append_trace(normal_trace, 1,2)
        interactive_fig.append_trace(diff_trace, 1,4)

    # create buttons
    for pathway in pathways_list:
        traces = [False] * len(interactive_fig.data)
        oldnum = currnum
        currnum += 3
        traces[0:oldnum] = [False for i in traces[0:oldnum]]
        traces[oldnum:currnum] = [True for i in traces[oldnum:currnum]]
        pathway_buttons.append(dict(label=str(pathway),
                                   method="update", args=[{"visible":traces}]))

    interactive_fig.update_layout(updatemenus=[dict(active=0,
                                                    buttons=pathway_buttons,
                                                    direction="down", pad={"r": 0, "t": 10},
                                                    showactive=True, x=0, xanchor="left", y=1.14, yanchor="top")],
                                 showlegend=False, width=700, height=700, title_text= "Compare Correlation", title_x=0.6, title_y=0.999)

    interactive_fig.update_layout(coloraxis = {'colorscale':"RdBu_r", 'showscale':True}, coloraxis2 = {'colorscale':"Reds", 'showscale':True})
    interactive_fig.layout.coloraxis.colorbar.x = 0.6
    interactive_fig.layout.coloraxis.colorbar.thickness = 15
    interactive_fig.layout.coloraxis.colorbar.tickfont = dict(size=10)
    interactive_fig.layout.coloraxis2.colorbar.x = 0.99
    interactive_fig.layout.coloraxis2.colorbar.thickness = 15
    interactive_fig.layout.coloraxis2.colorbar.tickfont = dict(size=10)
    interactive_fig.update_yaxes(showticklabels=False, row=1, col=2)
    interactive_fig.update_yaxes(showticklabels=False, row=1, col=4)
    interactive_fig.show()

In [ ]:
interactive_visualization()

## Final Analysis

In [ ]:
def final_analysis():
    sns.set(style="darkgrid", rc={'figure.figsize':(8.7,14)})
    simplified_pathways = cleaned_pathways[["Pathway Name", "FGGY", "NADK", "DISP1", "ENSG00000152022", "POS"]].set_index("Pathway Name")
    simplified_pathways = simplified_pathways.sort_values(["POS"], ascending=True)
    ax = simplified_pathways[["FGGY", "NADK", "DISP1", "ENSG00000152022"]].plot(kind="barh", stacked="True", figsize=(8, 14))
    ax.set_xlabel("POS Value")
    ax.set_title("Pathway Overlap Score Value Comparison", fontsize=15)
    ax.text(0.5, -3.5, "Figure 5. Compare the pathway overlap score by each of the four core genes.", ha="center")

In [ ]:
final_analysis()

### Relevance of Galactose Matabolism Pathway to Melanocytic Tumor 
#### Relevent Scientific Articles
**β-Galactosylceramidase Promotes Melanoma Growth via Modulation of Ceramide Metabolism** [(Mirella et al., 2020)](https://aacrjournals.org/cancerres/article/80/22/5011/645877/Galactosylceramidase-Promotes-Melanoma-Growth-via)
- Role of **sphnomyelinase** in inducing cancer: Ceramide, a basic strctural units of spingolipids, act as a tumor suppressor. It is known that defects in ceramide metabolism contribute to tumor cell survival. 
- β-Galactosylceramidase (GALC) is a lysosomal hydrolase that cataluzes the removal of β-galactose from β-galactosylceramide and other sphingolipids. 
- In this study, researchers successfully showed that downregulation of GALC leads to melanocyte differentiation. Therefore, researchers suggest that GALC may play an oncogenic role in melanoma by modulating the levels of intracellular ceramide. 
 